In [1]:
# import the necessary libraries
import os
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, PyMuPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

c:\Users\ASUSK5~1\miniconda3\envs\medchatbot\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [ ]:
# enter pinecone api key
PINECONE_API_KEY = "ADD YOUR PINECONE API KEY HERE"
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

In [3]:
# Initialize Pinecone client
pc = pinecone.Pinecone()

# list existing indexes
existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]
# existing_indexes

In [6]:
# declare pinecone index name
mcbot_idx_name = "medical-chatbot"

cloud = os.getenv("PINECONE_CLOUD", "aws")
region = os.getenv("PINECONE_REGION", "us-east-1")

spec = pinecone.ServerlessSpec(cloud=cloud, region=region)

# check whether the index already exists
if mcbot_idx_name not in existing_indexes:
    # create index
    # dimenstion is set based the embedding model and the dimension of its vectors
    pc.create_index(name=mcbot_idx_name, dimension=384, metric="cosine", spec=spec)

    while not pc.describe_index(mcbot_idx_name).status["ready"]:
        # wait until index is ready
        time.sleep(1)
    
    print(f"Index {mcbot_idx_name} has been successfully created.")
else:
    print(f"Index {mcbot_idx_name} already exists!")

# connect to the index
index = pc.Index(mcbot_idx_name)

Index medical-chatbot already exists!


In [7]:
# load data from the pdf file
def load_pdf(data_directory):
    loader = DirectoryLoader(data_directory,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)

    documents = loader.load()

    return documents

In [8]:
# read data
data_directory = "./data"
documents = load_pdf(data_directory)
# print(documents)

In [9]:
# create text splitter
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)

In [10]:
# split documents
# text_chunks = text_splitter.split_documents(documents)

Or you can simply declare a function performing as text splitter integrating all in one.

In [ ]:
# build a function to split text
def text_splitter(documents):
    rec_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,
                                                chunk_overlap=20)
    chunks = rec_splitter.split_documents(documents)

    return chunks

In [12]:
# generate text chunks through text_splitter()
text_chunks = text_splitter(documents)
len(text_chunks)

3006

In [13]:
# build a function to download model
def download_huggingface_embedding(model_name):
    embedding = HuggingFaceEmbeddings(model_name=model_name)
    return embedding

In [14]:
import sys
print(sys.executable)

c:\Users\ASUSK5~1\miniconda3\envs\medchatbot\python.exe


In [15]:
# download embeddings
model_name="sentence-transformers/all-MiniLM-L6-v2"
embedding = download_huggingface_embedding(model_name)

C:\Users\ASUS K513EQ\AppData\Local\Temp\ipykernel_17108\510078240.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embedding = HuggingFaceEmbeddings(model_name=model_name)


Now that we have the embedding model landed on our system, nobody would mind making a query in order to check whether it is working fine or not.

In [16]:
test_result = embedding.embed_query("Bo0om!")
test_result

[-0.06105484440922737,
 -0.010957086458802223,
 -0.06388892233371735,
 -0.012413165532052517,
 -0.06348859518766403,
 -0.018615080043673515,
 0.1121727004647255,
 0.01997370272874832,
 -0.09354047477245331,
 -0.06091571971774101,
 -0.030002398416399956,
 -0.01798173598945141,
 0.01098799891769886,
 -0.07747738808393478,
 -0.08996541798114777,
 0.06906592100858688,
 0.016005441546440125,
 0.039849694818258286,
 0.010348579846322536,
 0.016146862879395485,
 0.005521825514733791,
 -0.02962055429816246,
 0.049603309482336044,
 0.09224578738212585,
 -0.0312267504632473,
 0.014108521863818169,
 0.04687662795186043,
 0.09415562450885773,
 -0.0038906997069716454,
 -0.07334808260202408,
 0.07317619770765305,
 0.17730604112148285,
 0.0424121655523777,
 -0.0627225786447525,
 -0.007511781062930822,
 0.024614213034510612,
 -0.005599008407443762,
 -0.09019031375646591,
 0.015178652480244637,
 -0.028609750792384148,
 -0.02013222686946392,
 0.028419144451618195,
 -0.04579474404454231,
 -0.001043327269

**Yaaaay!** It is apprently working alright... then let's continue.
In fact the query is transformed into a vector as an embedding!

In [17]:
# Create a VectorScore from the text chunks already generated
# through which you can add more records to Pincone index

from langchain_pinecone import PineconeVectorStore

# Check if the index already hosts data
# therefore you won't have to upload it from scratch
# index = pinecone.Index(mcbot_idx_name)
index_stats = index.describe_index_stats()
existing_vector_count = index_stats["total_vector_count"]

if existing_vector_count == 0:
    print("Index is empty. Uploading new documents...")
    
    vectorstore_from_chunks = PineconeVectorStore.from_documents(
    text_chunks,
    index_name=mcbot_idx_name,
    embedding=embedding
    )
    
    print("Upload complete.")
else:
    print(f"Index already contains {existing_vector_count} vectors. Skipping upload.")
    # load the existing vector store whether or not new data was added
    vectorstore_from_chunks = PineconeVectorStore(index_name=mcbot_idx_name, embedding=embedding)
    print(f"Existing vectorstore is now loaded.")

Index already contains 3006 vectors. Skipping upload.
Existing vectorstore is now loaded.


In [18]:
print(vectorstore_from_chunks)
type(vectorstore_from_chunks)

langchain_pinecone.vectorstores.PineconeVectorStore

In [32]:
# perform a similarity search
query = "How do you define Ultrasound Technology?"
docs = vectorstore_from_chunks.similarity_search(query)
docs

[Document(metadata={'page': 14.0, 'source': 'data\\THE_GALE_ENCYCLOPEDIA_of_MEDICINE_SECOND_EDITION.pdf'}, page_content='Abdominal aorta ultrasound see Abdominal\nultrasound\nAbdominal aortic aneurysm see Aortic\naneurysm\nAbdominal hernia see Hernia\nAbdominal thrust see Heimlich maneuver\nAbdominal ultrasound\nDefinition\nUltrasound technology allows doctors to “see”\ninside a patient without resorting to surgery. A transmit-\nter sends high frequency sound waves into the body,\nwhere they bounce off the different tissues and organs to\nproduce a distinctive pattern of echoes. A receiver\n“hears” the returning echo pattern and forwards it to a\ncomputer, which translates the data into an image on a\ntelevision screen. Because ultrasound can distinguish\nsubtle variations between soft, fluid-filled tissues, it is\nparticularly useful in providing diagnostic images of the\nabdomen. Ultrasound can also be used in treatment.\nPurpose\nThe potential medical applications of ultrasound\nwer

`docs` retrieved above through `similarity_search()` is actually our knowledge base.

In [33]:
# retrive the page content of the document ranked first as the most related
docs[0].page_content

'Abdominal aorta ultrasound see Abdominal\nultrasound\nAbdominal aortic aneurysm see Aortic\naneurysm\nAbdominal hernia see Hernia\nAbdominal thrust see Heimlich maneuver\nAbdominal ultrasound\nDefinition\nUltrasound technology allows doctors to “see”\ninside a patient without resorting to surgery. A transmit-\nter sends high frequency sound waves into the body,\nwhere they bounce off the different tissues and organs to\nproduce a distinctive pattern of echoes. A receiver\n“hears” the returning echo pattern and forwards it to a\ncomputer, which translates the data into an image on a\ntelevision screen. Because ultrasound can distinguish\nsubtle variations between soft, fluid-filled tissues, it is\nparticularly useful in providing diagnostic images of the\nabdomen. Ultrasound can also be used in treatment.\nPurpose\nThe potential medical applications of ultrasound\nwere first recognized in the 1940s as an outgrowth of the\nsonar technology developed to detect submarines during'

At this point, it could be a good practice to have LLMs involved in order to produce a more sophisticated response.

In [34]:
# create a prompt template
prompt_template = """
Answer the question below given the context. Please don't try to make up one if you don't know the answer.

Question: {question}
Context: {context}

I only need a genuine and reliable answer. Thank you.
"""

In [35]:

medical_prompt = PromptTemplate(template=prompt_template,
                                         input_variables=["question", "context"])

In [36]:
chain_type_kwargs={"prompt": medical_prompt}

In [37]:
llm = CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                    model_type="llama",
                    config={"max_new_tokens": 512,
                            "temperature": 0.8})

In [38]:
retriever = vectorstore_from_chunks.as_retriever(search_kwargs={"k": 2})
print(type(retriever))  # Should print a valid retriever type

<class 'langchain_core.vectorstores.base.VectorStoreRetriever'>


**NOTE!** Before running the code block below, make sure that `langchain-community` is installed and up-to-date.

In [39]:
# QA pipeline
qa = RetrievalQA.from_chain_type(
                llm=llm,
                chain_type="stuff",
                retriever=retriever,
                return_source_documents=True,
                chain_type_kwargs=chain_type_kwargs)

In [40]:
# let us test the QA pipeline
result = qa.invoke(query)

In [42]:
# print(result["result"])